In [23]:
import pandas as pd
import torch
from torch.autograd import Variable
import os
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import random
from torch.autograd import Variable

In [2]:

tracknames = os.listdir("/home/jasper-ubuntu/Documents/Studie/Master AI/Computational intelligence/Torcs-CI/CI/train_data/")
# tracknames = ['aalborg.csv']
datadictin = {}
datadictout = {}
datadict = {}
framesin =[]
framesout=[]
data = []

for i, file in  enumerate(tracknames):
    path = os.path.join('train_data', file)
    print(path)
    datadictin[i] = pd.read_csv(path,sep=';', index_col=False).iloc[:-1, 3:]
    datadictout[i] = pd.read_csv(path,sep=';', index_col=False).iloc[:-1, 0:3]
    datadict[i]=pd.read_csv(path,sep=';', index_col=False).iloc[:,:]
    #Todo skip
    if datadict[i].iloc[-1,9] != 1:
        print('Not first place' )
    

#Concatenate all datasets
for i,file in enumerate(tracknames):
    framesin.append(datadictin[i])
    framesout.append(datadictout[i])
    data.append(datadict[i])
    
inp = pd.concat(framesin) 
outp = pd.concat(framesout) 
train = pd.concat(data)

#Drop missing values
inp.replace('', np.nan, inplace=True)
inp.dropna(inplace=True)

outp.replace('', np.nan, inplace=True)
outp.dropna(inplace=True)

train.replace('', np.nan, inplace=True)
train.dropna(inplace=True)
train.head()

train_data/road_alpine-1_7cars_2475231.csv
train_data/road_alpine-1_7cars_2475119.csv
train_data/road_alpine-1_7cars_2481619.csv
train_data/road_alpine-1_7cars_2481544.csv
train_data/road_alpine-1_7cars_2475043.csv
train_data/road_alpine-1_7cars_2475155.csv
train_data/road_alpine-1_7cars_2481731.csv
train_data/road_alpine-1_7cars_2481655.csv
train_data/road_alpine-1_7cars_2481432.csv
train_data/road_alpine-1_7cars_2481842.csv


,accel,brake,steer,angle,curLapTime,distFromStart,distRaced,gear,lastLapTime,racePos,...,oppos26,oppos27,oppos28,oppos29,oppos30,oppos31,oppos32,oppos33,oppos34,oppos35
0,1.0,0.0,0.0,3.019920e-07,-0.982,6306.65,0.0,0,0.0,7,...,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0
1,1.0,0.0,0.0,3.019920e-07,-0.962,6306.65,0.0,0,0.0,7,...,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0
2,1.0,0.0,0.0,3.019920e-07,-0.942,6306.65,0.0,0,0.0,7,...,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0
3,1.0,0.0,0.0,3.019920e-07,-0.922,6306.65,0.0,0,0.0,7,...,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0
4,1.0,0.0,0.0,3.019920e-07,-0.902,6306.65,0.0,0,0.0,7,...,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0


In [3]:
input_size = inp.shape[1]
output_size = outp.shape[1]
datapoints = inp.shape[0]
print(datapoints,input_size,output_size)

331770 72 3


In [4]:
# Hyper Parameters
input_size = inp.shape[1]
hidden_size = 50
output_size = outp.shape[1]
num_epochs = 20
batch_size = 100

In [26]:

# Neural Network Model
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h = F.tanh(self.fc1(x))
        h = F.tanh(self.fc2(h))
        h = F.tanh(self.fc3(h))
        return h
    
    
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h = F.tanh(self.fc1(x))
        h = F.tanh(self.fc2(h))
        h = F.tanh(self.fc3(h))
        return h

class RNNJens(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(RNNJens, self).__init__()
        self.hidden_dim = H
        
        self.recurrent = nn.GRU(D_in, H)
        self.linear = nn.Linear(H, D_out)
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        """
        Before we've done anything, we dont have any hidden state.
        Refer to the Pytorch documentation to see exactly
        why they have this dimensionality.
        The axes semantics are (num_layers, minibatch_size, hidden_dim)
        """
        return (torch.autograd.Variable(torch.zeros(1, 1, self.hidden_dim)),
                torch.autograd.Variable(torch.zeros(1, 1, self.hidden_dim)))
        
    def forward(self, x):
        """
        In the forward function we accept a Variable of input data and we must return
        a Variable of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Variables.
        """
        lstm_out, self.hidden = self.recurrent(x, self.hidden)
        y = self.linear(lstm_out)
        y_pred = F.tanh(y)
        
        return y_pred

In [6]:
# def minibatch(data, batch_size=32):
#     for i in range(0, len(data), batch_size):
#         yield data[i:i+batch_size]

In [7]:
def get_variable(data, labels = False):
    """Get a Variable given data (pandas dataframe)"""
    # Use float (not double) for better precission: (see reply of smth)
    # https://discuss.pytorch.org/t/problems-with-target-arrays-of-int-int32-types-in-loss-functions/140/3
    
    # 'Labels' or 'objective' need to be long to work (for cross entropy, for MSE they both need to be long)
    if labels:
        tensor = torch.from_numpy(data.iloc[:,:].as_matrix()).float()
    else:
        tensor = torch.from_numpy(data.iloc[:,:].as_matrix()).float()
#     print(tensor)
    return Variable(tensor)

In [8]:
def preprocess(batch):
    """ ... """

    x = batch.iloc[:-1, 3:]
    y = batch.iloc[:-1, 0:3]

    return x, y

In [13]:
model = RNNJens(input_size, hidden_size, output_size)
# rnn_jens = (input_size, hidden_size, output_size)

In [14]:
#TODO learning rate scheme?


In [18]:
# NOT ADAM because we have no stochastic objective function: https://arxiv.org/abs/1412.6980

optimizer = optim.SGD(model.parameters(), lr=0.001)

In [19]:
for ITER in range(num_epochs):
    
    # Take a random sample of size batchsize
    
    train_loss = 0.0
    start = time.time()
    updates = 0

    for i in range(int(datapoints/(batch_size))):
        batch = train.sample(n = batch_size)
        updates += 1

        # pad data with zeros
        x, y = preprocess(batch)
        
        # forward pass
        scores = model(get_variable(x))
        targets = get_variable(y, labels= True)
        loss = nn.MSELoss()
        output = loss(scores, targets)
        train_loss += output.data[0]

        # backward pass
        model.zero_grad()
        output.backward()

        # update weights
        optimizer.step()

    print("iter %r: avg train loss=%.4f, time=%.2fs" %
          (ITER, train_loss/updates, time.time()-start))

    # evaluate
#     _, _, acc_train = evaluate(model, train)
#     _, _, acc_dev = evaluate(model, dev)
#     print("iter %r: train acc=%.4f  test acc=%.4f" % (ITER, acc_train, acc_dev))

RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.

In [27]:
# Training cell for LSTM
for ITER in range(1):
    
    # Take a random sample of size batchsize
    
    train_loss = 0.0
    start = time.time()
    updates = 0

    for i in range(int(datapoints/(batch_size))):
        batch = train.sample(n = batch_size)
        updates += 1
        
        # Also, we need to clear out the hidden state of the LSTM,
        # detaching it from its history on the last instance.
        model.hidden = model.init_hidden()

        # pad data with zeros
        x, y = preprocess(batch)
        
        # forward pass
        scores = model(get_variable(x))
        targets = get_variable(y, labels= True)
        loss = nn.MSELoss()
        output = loss(scores, targets)
        train_loss += output.data[0]

        # backward pass
        model.zero_grad()
        output.backward()

        # update weights
        optimizer.step()

    print("iter %r: avg train loss=%.4f, time=%.2fs" %
          (ITER, train_loss/updates, time.time()-start))

    # evaluate
#     _, _, acc_train = evaluate(model, train)
#     _, _, acc_dev = evaluate(model, dev)
#     print("iter %r: train acc=%.4f  test acc=%.4f" % (ITER, acc_train, acc_dev))

iter 0: avg train loss=0.0526, time=336.65s


In [22]:
# Test prediction
testval = get_variable(x)
print(testval)
print(model(testval))
print(y)

Variable containing:
     0.2130     80.3540   3242.7800  ...     200.0000    200.0000    200.0000
    -0.2783    106.4880   5923.4800  ...     200.0000    200.0000    200.0000
    -0.0297      2.2060    150.9400  ...     200.0000    200.0000    200.0000
                ...                   ⋱                   ...                
     0.0571     82.4100   4065.5801  ...     200.0000    200.0000    200.0000
     0.0670     69.7460   3424.3799  ...     200.0000    200.0000    200.0000
     0.0503     25.9160    865.6350  ...     200.0000    200.0000    200.0000
[torch.FloatTensor of size 99x72]

Variable containing:
(0 ,.,.) = 
  0.8390  0.0279  0.0725

(1 ,.,.) = 
  0.8088  0.0896  0.0939

(2 ,.,.) = 
  0.8687  0.0073  0.0061

(3 ,.,.) = 
  0.8680  0.0148  0.0179

(4 ,.,.) = 
  0.8610  0.0258 -0.0206

(5 ,.,.) = 
  0.8614  0.0249 -0.0226

(6 ,.,.) = 
  0.8655 -0.0617 -0.0268

(7 ,.,.) = 
  0.8554 -0.0244  0.0714

(8 ,.,.) = 
  0.8601  0.0231 -0.0249

(9 ,.,.) = 
  0.7345  0.0329 -0.090

In [40]:
save_path_model = 'simple_ff_model_20epochs_batch100.pt'
save_path_dict = 'simple_ff_DICT_20epochs_batch100_dict_hiden_size50_2HL_INP72_outp3_Forward_tanh_tanh_tanh.pt'
# https://stackoverflow.com/questions/42703500/best-way-to-save-a-trained-model-in-pytorch
torch.save(model, save_path_model)
torch.save(model.state_dict(),save_path_dict)
# model.save_state_dict('trainedmodel

/home/jasper-ubuntu/miniconda3/lib/python3.6/site-packages/torch/serialization.py:147: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
